In [22]:
import numpy as np
from numba import jit
import matplotlib

class EM_algorithm(object):
    def __init__(self,Q,Q_0,w, seq_array):
        
        self.Q = Q
        self.Q_0 = Q_0
        self.w = w
        self.seq_array = seq_array
        self.number_of_sequences = len(self.seq_array[:])
        self.sequence_length = len(self.seq_array[1][:])
        
    def get_Q_row(self, c):
        if c == 'A':
            return 0
        elif c == 'C':
            return 1
        elif c == 'G':
            return 2
        elif c == 'T':
            return 3
        else:
            print("Not valid")
            return
    
    def z_ij(self):
        z = np.array([[1 for x in range(self.sequence_length)] for y in range(self.number_of_sequences)], dtype=np.double)
        
        for i in range(self.number_of_sequences):
            
            ith_sequence = self.seq_array[i][:]
            den = self.denominator(ith_sequence)
            
            for j in range(self.sequence_length - self.w + 1):
                
                num = self.numerator(ith_sequence,j)
                z[i,j] = np.divide(num,den)
        
        self.z = z
                
        return self.z
    
    def numerator(self,ith_sequence,j):
        scale_factor = np.double(40)
        prod1 = np.double(1)
        prod2 = np.double(1)
        prod3 = np.double(1)
        
        if j>0:
            for row in range(0,j):
                q_row = self.get_Q_row(ith_sequence[row])
                prod1 *= np.multiply(self.Q_0[q_row,0],scale_factor)
        else:
            q_row = self.get_Q_row(ith_sequence[0])
            prod1 *= np.multiply(self.Q_0[q_row,0],scale_factor)
            

        i = 1
        for row in range(j,j+w-1):
            q_row = self.get_Q_row(ith_sequence[row])
            prod2 *= np.multiply(self.Q[q_row,i],scale_factor)
            i += 1
        
        for row in range(j+w,self.sequence_length):
            q_row = self.get_Q_row(ith_sequence[row])
            prod3 *= np.multiply(self.Q_0[q_row,0],scale_factor)
            
        return np.multiply(np.multiply(prod1,prod2),prod3)
    
    def denominator(self, ith_sequence):
        accumulator = np.double(0)
        for k in range(self.sequence_length - self.w):
            accumulator += self.numerator(ith_sequence,k)
        return accumulator 
                
        return accumulator
    def update_Q_z(self,num_updates):
        self.z = self.z_ij()
        
        for i in range(num_updates):
            self.Q = self.Q_estimate()
            self.z = self.z_ij()     
        return
    def Q_estimate(self):
        q_est = np.array([[0 for x in range(self.w)] for y in range(4)])
        den = self.q_denominator()
        chars = ['A','C','G','T']
        
        for k in range(self.w):
            for b in range(4):
                num = self.q_numerator(k,chars[b])
                q_est[b,k] = np.divide(num,den)
                
        return q_est
    
    # Check here for errors.... k and j might be mixed up
    def q_denominator(self):
        accumulator = np.double(0)
        for i in range(self.number_of_sequences):
            for j in range(self.sequence_length - self.w):
                accumulator += self.z[i,j]
                
        return accumulator
    
    def q_numerator(self,k,c):
        accumulator = np.double(0)
        for i in range(self.number_of_sequences):
            seq_of_interest = self.seq_array[i][:]
            for j in range(self.sequence_length - self.w):
                if seq_of_interest[j + k - 1] == c:
                    accumulator += self.z[i,j]
                    
        return accumulator
#    def string_EM(self,num_similar):
#        function [ s ] = string_EM( seq_array, Acon,w, num_similar )
#        index = 1;
#        for row = 1:num_sequences
#        seq_interest = seq_array(row,:);
#        [val, ind] = sort(Acon(row,:), 'descend');
#        for i = 1:num_similar
#        
#       s(index,:) = seq_interest(ind(i):(ind(i) + w -1));
#        index = index + 1;
        


In [23]:
w = 7
Q = np.array([[0.25 for x in range(w)] for y in range(4)], dtype=np.double)
Q_0 = np.array([[0.25 for x in range(1)] for y in range(4)], dtype=np.double)

In [24]:
seq_array1 = ['AAAGTAGAAAACTTGAGCACTTATTTCCTGCGCATGTCATATGTTAATTTTCCTCAACTGCGCTGAATACGTCCTGTCAATTCAAATATATCACGTTGTGAGCAGCCCTGAAGAAGAAAACCTCAACAGCAGTATTACTATTACAATCAAACAACTTTAGTGCCGCGTGATACCGGGGGTTGAAGTGGGTGCATTGAGCCGTATTCTTCTTCCCCGTAAGAAAGTTGTGTATCCTTTTTACTGCGTTGTAATAGCTTCTGAAAACCTAAAAAATGAACGCTATGTAGCTCATATCCGTTTTGCATAAGTAAGAATAACTACTTGTGCAGGGTGCCGAAAGGGATGGAAAACCGCTGCAGCAACCCTTGTTACATACAGTCGGATCCATCTGACTTACTTTCCTTGCGTCTCCCTGCGCGATTTTGTTGGCCATTTTCCAGATCCTCTAGAATTTTTCAAGGGTCGAGCCGTAGGAGGATTCTCTCAGAAGGCAAAAACGCATCGAAAGCGTGCTTTGTAAGAATATTTGGTATGGCTAAAGTAAGCAAAGCCATATCCCGATCCCGATCCCGACTCTTATTCCGATCCCTTCCGCCACATCCTGCATGTTTATTCGAATACCAAATTAGCTCATCTTCGTTATTTCATCATCCCTTTCTGCTATGGCAAGGACAAGTTTTTTTCTAGCATCTCATCGAAAACTTTCCTCTCCCTAATTGGCCAAAGTTTTCATATTCATCATCAGTTAGAAAGTATAATATCAATCCCTTACCTCATTACAAGTTGTATCACACTAAAAAAATCATATATAAGTCTGTGAGAGTCTTCAATTATTTAGCGTAACACCTATTCACTTTCTAATCTTGTTTCTTGTTTTTACATTCTGCAATACAACACAACAACAAATATTAACTCAATTATTATTATTTATAATTACAAAAACAAAACAACAAGTTTGAGACTTTAATATCTTTTGATTACTAAAAACAACAAATTTCAA',
              'ACCTGTAACTATGTTGGCACAAACGAAAAAAGTTTTGAGCTTAAGCACGCCTGAAAAGAAAATGTCCAAGAGTGATCCGAATCACGACTCTGTAATTTTTTTGAATGATGAGCCTAAGGCAATTCAGAAAAAGATTAGAAAAGCTTTAACAGATTCCATTTCTGATAGGTTCTACTACGACCCTGTAGAAAGACCAGGCGTATCGAATTTAATTAACATTGTCAGTGGCATTCAAAGAAAATCGATTGAAGATGTCGTAGAAGATGTATCTCGTTTCAATAACTATAGGGATTTCAAAGATTATGTTTCAGAAGTGATAATTGAGGAATTGAAAGGCCCAAGAACAGAATTTGAGAAATATATCAACGAACCGACCTATTTGCATAGTGTCGTTGAATCTGGCATGCGCAAAGCGAGAGAAAAAGCAGCAAAAAACCTGGCCGACATTCATAAAATAATGGGCTTCTGACTGTCCCCGGCTTTAGGCTGCTGCAAACGCAATGTAAATAATAATACAGTTTAACTTGTACGTATCTTTGTTATTTCAGAATATGCAAGATTCCTCACTCGGCAGATCTCATCAATAAACGTTAGTATCTTGAAAAATACGCAGATGTTGGTACAGTCTCTTGCAGAGGGCCAGCACCGTTAGAATGTTGTGTTGCCACCGTCTGGCTCGAAAACAGGTCGATCTGTGTAAGTATTGTCCTGGGTGCAGTTATTTGTAGTAGCGATTTTCGCGGGCAGTGGAAAAGAACGGGTGCAGTGCGGCGATGAGTGCAATTAGCTGCCGAAGGGTTTGCACCGGTGATTCCGAGACCTTTTGCACACACTTGTATATATTTAGGGAAGTCACGGTAACTAATGGATTTCTTCTCAAATTTGAGACATATGGATTAGACTTTAATTCTGCCTGGTACTAAGCTACTTTTCAAAGGTATTTGAGATATTTCGTAAATTTTGCGGCCAAGGAGTTGAGGTGTTTAATACTTAAAACGAA',
              'GTGCAGACTAATGAAAAAGACCCCAGACCCATCTCCACCTTTTGCAAGCACGAAGAACGTAGGCATGTCAAACGAAGAACCGGAAAAAATGGTTAACGATCGAATTGTGGTGAAGGCCATCGAGCCAAAAGATGAGGAAGCCTGGAACAAACTGTGGAAGGAGTATCAGGGTTTCCAAAAAACGGTTATGCCTCCGGAAGTAGCCACCACTACCTTCGCAAGGTTTATAGACCCTACGGTCAAACTATGGGGTGCTCTAGCCTTTGACACCGAGACCGGCGATGCAATTGGCTTTGCACACTACCTGAACCATTTGACGTCGTGGCATGTCGAAGAAGTTGTCTACATGAATGACTTATATGTCACTGAACGCGCAAGAGTCAAAGGCGTAGGTAGAAAGCTCATAGAATTTGTATACAGCCGTGCCGATGAGCTCGGTACGCCTGCAGTGTACTGGGTTACAGACCATTACAATCACCGCGCACAGTTGCTGTACACCAAAGTGGCTTACAAGACAGATAAGGTTCTTTACAAGAGAAACGGATACTGAAACCGCTGTGTAATGTAGACTACTGTGTATAGAGATAACTCTAGCTTCTATAGGCTCACTTGCGAAGCTTTTTTCCCACTTTTTCCCAGTGAAATAGTTTCTTATTGGCTTTTCATAGTTTGTTCTCCGATTTAATCGCAAGCGCCGTACGGCGCAGGCACAGAATGACTTGCAGCTTAATCCAGGTGCATTTTAAGAAACTTATCTGCTGGGAAGACCCCGAAGTTCATCGTTTACAATGTGCCGTAAAGAACACATTTGAGGGTGCAATCTGGTTCGTCTTTTGCCATAATTAGTAGTGCTCATCCGTGATCCTTTAATTTACTTTTGTGATGCGATGATTTGCAGATACTATATAAGCAATGGACACCGTCCATCACCGATGAGGCTAGAACATCGTAGTGCTTTGTAAAAGAACTGTTAAAGACCCAGTACGAAAATTTTTCCATA',
              'AAGCCACTCGACAGCGTCTTCCAGTATAACGTTGACGAAGTCGTCGAAACCAACTAACGTGCCCTCGAACTCGCGGTTCGACTGCAGCACAATCAACACTTTCTGGTTAATTGTTTTATCTATGACTTCCAAAGGCAAAATCTCCGGTAGACTCATTTTGCGCGCTTCTGTGTATGCTTGTGAGTGCACGTGTATACGCTATGCTTGTTGTGAGACCATCTATCCGCCAAGGAAGCCTTTTTCGCTTTCTCTGTATTTTTTCTACTTCAATAAGAAAGCAAGTATTCTGTTTTGCTACCCGGATGCAAAGACAAAACCCGAGCATCCGGGTAATAAAAGAGCAGGGTCCGGCTGAAGGTGCCAAAAAAGACCAAAAAGGGCTATATTATCTCTCTCTTACGATTGCCTCTGGTTAAAAAATGAAATAGATAATAAAAGGGCAGCAAAGAAAAAAAAGAAGAATACAATATACGTATACGTTCTCAACAATTTTTCGTCCTGTTTTTTTCTTGACCACGGTCACTACTAACTCCTGAGACATCAGACCAACGTGTAGAAAAAAAGAAATCGGGTCCGGCACTGACACACGCTCTGCGCCAATCAACCAGGGCGCATCGAGCATTCTTTCCAGGCCAGGGTGCAGAAATACTATTATTAAGTGCGCGAATACTGCTGGTGCATTAAACGAGCGAGTACAGTGAGCCCGCCGCTGACTGCCGACTCACGCCAGGCCGCTATTTCCTTCCCCCGCCCGCGTAAGCCGTGCCGAGCACCGTCGCAGCAGTAAGAAGGTGCGCATTTCATTCTCTGCACAGCGTAGATGAATGCACCCGATTTCTAAGTTGGTGTTGCAAGGCCCGGTGGAAGCGATGGGCTGTAGTTCGACCTTGAGCTTTTAGTTATATAAAATGCTCAATGTGAGCAAAAAAATACATAACACGACGGATTGTACGTTGTCGAAATCTTCTCTCAGCAGGTCATCACACATATACTTCCCGCC',
              'CCAAACCTGAATCCTTAATTTACGGTACAGCGCCCGGTCATGTTCCCCGATTGCCTGTTTCAAAAACGATAGCAGCACGGAAATAACAATCACTGCTTCCAAGCACTCTCTGAACACAACGAAGAAAACGGCCACGTTAAACACTTTGTTAGGCATGGCGGGAAGTATATGTGTGATGACCTGCTGAGAGAAGATTTCGACAACGTACAATCCGTCGTGTTATGTATTTTTTTGCTCACATTGAGCATTTTATATAACTAAAAGCTCAAGGTCGAACTACAGCCCATCGCTTCCACCGGGCCTTGCAACACCAACTTAGAAATCGGGTGCATTCATCTACGCTGTGCAGAGAATGAAATGCGCACCTTCTTACTGCTGCGACGGTGCTCGGCACGGCTTACGCGGGCGGGGGAAGGAAATAGCGGCCTGGCGTGAGTCGGCAGTCAGCGGCGGGCTCACTGTACTCGCTCGTTTAATGCACCAGCAGTATTCGCGCACTTAATAATAGTATTTCTGCACCCTGGCCTGGAAAGAATGCTCGATGCGCCCTGGTTGATTGGCGCAGAGCGTGTGTCAGTGCCGGACCCGATTTCTTTTTTTCTACACGTTGGTCTGATGTCTCAGGAGTTAGTAGTGACCGTGGTCAAGAAAAAAACAGGACGAAAAATTGTTGAGAACGTATACGTATATTGTATTCTTCTTTTTTTTCTTTGCTGCCCTTTTATTATCTATTTCATTTTTTAACCAGAGGCAATCGTAAGAGAGAGATAATATAGCCCTTTTTGGTCTTTTTTGGCACCTTCAGCCGGACCCTGCTCTTTTATTACCCGGATGCTCGGGTTTTGTCTTTGCATCCGGGTAGCAAAACAGAATACTTGCTTTCTTATTGAAGTAGAAAAAATACAGAGAAAGCGAAAAAGGCTTCCTTGGCGGATAGATGGTCTCACAACAAGCATAGCGTATACACGTGCACTCACAAGCATACACAGAAGCGCGCAAA',
              'AAGGACAGAGAAATTCATATTAAGAGAGCTAGAACTCCAGGCCAAATGCAAAGAGGAGGTTTCAGAGGCAGAGGCGGTTTCAGAGGCAGAGGAGGTTTTAGAGGAGGTTTCAGAGGCGGCTACAGAGGAGGTTTCAGAGGCAGAGGGAACTTCAGAGGTAGAGGCGGCGCCAGAGGTGGTTTCAATGGACAAAAAAGGGAAAAGATTCCATTAGACCAAATGGAAAGATCAAAGGATACCTTATATATTAATAACGTCCCATTCAAAGCTACCAAAGAGGAGGTCGCTGAATTTTTCGGTACTGACGCCGACTCCATCTCTTTGCCAATGAGAAAAATGAGAGACCAACACACTGGTAGGATCTTCACATCCGATTCTGCTAATAGAGGTATGGCATTTGTCACTTTCAGTGGTGAAAACGTTGATATTGAAGCTAAAGCTGAAGAATTTAAAGGCAAGGTTTTCGGTGACAGGGAGTTAACTGTAGATGTTGCTGTTATTAGACCAGAAAATGATGAAGAAGAAATTGAGCAAGAAACTGGTTCTGAAGAAAAGCAAGAATAATTACTTCTTACCCACATCCCTATTTCTAACTTGAGTTTTTGCTAGAGTTTTGTATTTTTGTTCACCTTCCCTGCAAAAGAAATATGTGTATTTATATATGCGTGTATACCTATATATGATATGTAAAAATGAGACGCCCCTGTTTTATTTTCAAACACTTCCCCGTATAGTTTTTTGCAATGACACTACTTTAACTTCTTCGACATGATTTGCTTTAGCACTACGAAGGATTGCATCATAACGTTTCGAAAGGGGTGCACTTTTAAAAACCAGTAAGTGAGTGCCTCGTGAACTGCTATTTTCGTATTTTGAAAAAAAAAATAAAAAAAAACTCCCTTATATATATATATAAATATCTATGTACTAAATGTCAAATCGCTAGCTCTCACCTATATCTTATTCATGCTGCAACCTCAATGGTTCACTACCTCGGAGA',
              'ACAGAATTTCAAGAGTAGCTTTGCAGATAGCCATGTTGATGAGCAATACTTTGCGTACAAATACAACCTTTTGCAGACATCGTATATGGCCATGTGCACAGCTAATTTTTCTCACAATTGGTGCCAATTATATGTGGATATGAGATATCTAATAGAACGAGATGAAAAGCTGTATAGAATAAAAGAATTAACAAGGAACCTGCTGGAAACTAAACTGAACATGAAATATCGTATTGTCTGTCAACTAATCAGGCATCAATTGACCGAATTTCGTGAGAACGAGAGAAATCCATCGTGGGACGCCACCATTGAGAAACTGCTACCTTATATCTTGAAGGAAATTGTTCGTCCACTGCAGAAAATAAGAGGTGAAGAAGGTTCCCGTTACTTGTTGTCTTTCTTAAACTTTTTATATAACGATTGCGTGACAAAGGAAATATTAAAATGGCAGATTATCTCTGAGGTCAATTCCGAAAATTTGGGTGAGCTCGTATCTTTATTAGTGAACAACACCGATATACAATTATTAGCGAAGGAACCAAGTTACAAAAAGATGAGAGAAAAATTCGCCACTATGGGCAAGTTTCTACCATTACATTTAAAGGAGATTATGGAGATGTTTTACAATGGGGATTTTTATCTTTTTGCGACAGACGAACTAATCCAATGGATAGAGTTATTGTTTGCCGACACTCCCCTGCGAAGGAATGCCATTGATGATATTTACGAAATTAGAGGCACTGCTCTAGATGATTAAAAAATAATTAATCGTAGGATAGGTTACATGATCATTGCAATAGATGTACATCTTGTATTACGTATGCGTAAGTTGTCTTTTTTTCAGTTTTCTAATAAATTTCACGTGATGCGGTAGTTTTTTTTGGGGGTGCAAAGTTGGCACATCTTGTAGTGCATTTTTGAGCTTCTTAAATGACTTGAATCATCTTAAAACATTGCAGGATGAGATTTCCAACGACACAAGAGAGAACTAGCGCAAAAG',
              'TTCGTTATGCTGGACGCTCTCAAAACCCCCATGGCATTCACAGGTCTGATGGCCATCAGAGCAGGCCTTGCAAATCTCGTTATAACAGGAAGCATCAGGTTAAATATGTGTTATGTGTTGTGTTTTCCTTATTCAATTGTTATATGAAGGGATGTTCTTATGGATTTAAGAATAATCTCTGAACACAAATATATATGCGGAAGAGCGAAGGAAGAGAGCAGAAAAGGCTACAACGAAACCCTTTCATTTACTTATATAAATAGCCTGAGTCATTTTTCCTCTCAATTTTTGCATCATCATGATCATACATGCTTTTTTTTTCCTTCTCCATGGCTAATTGGGGTGCAACGCACCTTATCGCCTTTGACGGCCTTGGCTAAGCGGAGATAAAGCCTCCGAACTTCGGCCCCGAAGGGATTGGCGCGGATATACACGGAAATAGCTTGTCATTTTTCCTTCGGTTGATGTCCGCGGACCGAGAAGTGGTGTTTCCACATTACGTCTCAGTGATGCAATCATCATGGCACGGGCATTAATTAAGGTTAAGGAACCACTTCTCTACGCCATACTCCAAACAATAACAATATTAGACCTAGCCAATTGATACCCATGAGCGTGCAGTCATTGCCATGGATTATTCATCTCTTTTCGCTGTTCTCTTTCTTTTCAAGTTTGTCATCATCATGCCTTCACTTTTGCCTTTCCATCTTTCTTTTTGCTGCAAAATAAAGGGAAGAGGGGTAAAAACGCAAGGAAGAACAAGAAGAAGAGGGTAGTGCAAGAAAAAGAAAAGAAGAAAAAAAAAAAAAAGTAATCTTGATACCGTGAGCAAATAAGCTAACGGAAAGCGTAAGAAAGAAGAGCGTGTTTTGGGAAATAACACCACAGCATAAAGCTAAAATTCAGTTTATATAATCTATAGTAGTCCTATAGAAATTGCGAATAACGGAAACAATAGTCCACCAAAGCAAGCATAGGGAGTGGAGATAGCATCTAGGTT'
              ];

In [25]:
em_test = EM_algorithm(Q,Q_0,w,seq_array1)

In [26]:
em_test.z_ij()
em_test.z

/Users/Versace/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:68: RuntimeWarning: overflow encountered in double_scalars
/Users/Versace/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:70: RuntimeWarning: overflow encountered in multiply
/Users/Versace/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: overflow encountered in double_scalars
/Users/Versace/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


array([[ nan,  nan,  nan, ...,   1.,   1.,   1.],
       [ nan,  nan,  nan, ...,   1.,   1.,   1.],
       [ nan,  nan,  nan, ...,   1.,   1.,   1.],
       ..., 
       [ nan,  nan,  nan, ...,   1.,   1.,   1.],
       [ nan,  nan,  nan, ...,   1.,   1.,   1.],
       [ nan,  nan,  nan, ...,   1.,   1.,   1.]])

In [64]:
em_test.number_of_sequences

8

In [60]:
len(seq_array1[:])

8

In [36]:
len(em_test.z[:,0])

8

In [29]:
#em_test.update_Q_z(2)
np.double(2)
np.multiply(np.double(2),np.double(2))
np.power(np.double(1.25),1000)

8.128548625557735e+96